In [186]:
import pandas as pd
import gradio as gr


def get_BFCs(results_df):
    df = results_df[['id', 'commit_hash', 'is_bug_fixing_commit']]
    df = df.rename(columns={'commit_hash': 'hash', 'is_bug_fixing_commit': 'BFC'})
    df['hash'] = df['hash'].str.slice(0, 10)
    return df

results_A = pd.read_csv('130_results_michel.csv')
results_B = pd.read_csv('130_results_abhishek.csv')
results_C = pd.read_csv('130_results_david.csv')

BFCs_A = get_BFCs(results_A)
BFCs_B = get_BFCs(results_B)
BFCs_C = get_BFCs(results_C)

BFCs = pd.merge(BFCs_A, BFCs_B, on='hash', how='inner', suffixes=('A', 'B'))

BFCs = pd.merge(BFCs, BFCs_C, on='hash', how='inner', suffixes=('', 'C'))
BFCs = BFCs[['hash', 'BFCA', 'BFCB', 'BFC']]
BFCs = BFCs.rename(columns={'BFC': 'BFCC'})
BFCs['hash'] = BFCs['hash'].str[:10]

In [187]:
total_commits = len(BFCs)
BFCs_R = pd.read_csv('130_review.csv')
BFCs = pd.merge(BFCs, BFCs_R, on='hash', how='inner')
BFCs['BFCR'] = BFCs['bfc']
BFCs_disagree = BFCs.query("(BFCA != BFCB) or (BFCA != BFCC)")
total_disagree = len(BFCs_disagree)

In [195]:
print(f"Total commits: {total_commits}")
print(f"Total disagreement: {total_disagree}")

persons = ['A', 'B', 'C', 'R']
print("    ", "    ".join(persons))
for row in persons:
    print(f"{row}", end=" ")
    for column in ['A', 'B', 'C', 'R']:
        if column > row:
            dis = len(BFCs_disagree.query(f"(BFC{column} != BFC{row})"))
            print(f"{dis:>4}", end=" ")
        else:
            print("    ", end=" ")
    print()

persons = ['A', 'B', 'C', 'R']
print("      ", "      ".join(persons))
for row in persons:
    print(f"{row}", end=" ")
    for column in ['A', 'B', 'C', 'R']:
        if column > row:
            dis = len(BFCs_disagree.query(f"(BFC{column} != BFC{row})"))
            dis = (dis / total_commits)
            print(f"{dis:2.2%}", end=" ")
        else:
            print("      ", end=" ")
    print()


Total commits: 129
Total disagreement: 33
     A    B    C    R
A        30   15   13 
B             21   19 
C                   8 
R                     
       A      B      C      R
A        23.26% 11.63% 10.08% 
B               16.28% 14.73% 
C                      6.20% 
R                             
